In [ ]:
import glob
from paddleocr import PaddleOCR, draw_ocr
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import math
import imutils
import json

In [ ]:
image_paths = glob.glob("IndianDataset/img/*")

In [ ]:
image_paths[0]

In [ ]:
ocr = PaddleOCR(use_angle_cls=True)

In [ ]:
def unit_vector(vector):
    if np.sum(np.abs(vector)) == 0:
        return vector
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def findClockwiseAngle(self, other):
    unit_self = unit_vector(self)
    unit_other = unit_vector(other)
    # using cross-product formula
    return -math.degrees(math.asin((unit_self[0] * unit_other[1] - unit_self[1] * unit_other[0])))

def withinRange(values, num_std = 2):
    std = np.std(values)
    mean = np.mean(values)
    return (mean - num_std * std < values) & ( values < mean + num_std * std)

## Go through each file, detect tilted angle compared to x-axis. Rotate back to x-axis if needed

### Paddle OCR works even if text is rotated, but LayoutLM doesn't like rotated bounding box
### So if a box is detected in a rotated fashion, all its edges will be expand to its projected vertical and horizontal axis

<img src="doc_imgs/bounding_box.png">

In [ ]:
df = pd.DataFrame(columns=["x0", "y0", "x1", "y1","x2", "y2", "x3", "y3", "conf", 'text', 'avg_ang'])

rotate_angles = {} # save a dict of key-value of filename-tilted angle 

for path in image_paths:
    file_name = path.split('\\')[-1].split('.')[0]
    image = cv2.imread(path)
    result = ocr.ocr(path)
    boxes = [line[0] for line in result[0]]
    txts = [str(line[1][0]) for line in result[0]]
    scores = [line[1][1] for line in result[0]]
    d = pd.DataFrame({'x0':[i[0][0] for i in boxes], 'y0':[i[0][1] for i in boxes],'x1':[i[1][0] for i in boxes],'y1':[i[1][1] for i in boxes],'x2':[i[2][0] for i in boxes],'y2':[i[2][1] for i in boxes],'x3':[i[3][0] for i in boxes],'y3':[i[3][1] for i in boxes], 'conf': scores, 'text':txts, 'avg_ang': np.nan})
    # for col in d.columns[:-2]:
    #     d[col] = d[col].astype(int) # remember that SROIE2019 dataset requires all fields to be int
    
    for i in range(len(d)):
        hor1 = (d.loc[i, 'x1'] - d.loc[i, 'x0'],
                d.loc[i, 'y1'] - d.loc[i, 'y0'])
        hor2 = (d.loc[i, 'x2'] - d.loc[i, 'x3'],
                d.loc[i, 'y2'] - d.loc[i, 'y3'])
        ver1 = (d.loc[i, 'x3'] - d.loc[i, 'x0'],
                d.loc[i, 'y3'] - d.loc[i, 'y0'])
        ver2 = (d.loc[i, 'x2'] - d.loc[i, 'x1'],
                d.loc[i, 'y2'] - d.loc[i, 'y1'])

        ang1 = findClockwiseAngle(hor1, (1, 0))
        ang2 = findClockwiseAngle(hor2, (1, 0))
        ang3 = findClockwiseAngle(ver1, (0, 1))
        ang4 = findClockwiseAngle(ver2, (0, 1))

        avg_ang = np.average([ang1, ang2, ang3, ang4])
        d.loc[i, 'avg_ang'] = avg_ang
        Xs = d.loc[i, ['x0', 'x1', 'x2', 'x3']]
        Ys = d.loc[i, ['y0', 'y1', 'y2', 'y3']]
        d.loc[i, 'x0'] = Xs.min()
        d.loc[i, 'x3'] = Xs.min()
        d.loc[i, 'x1'] = Xs.max()
        d.loc[i, 'x2'] = Xs.max()

        d.loc[i, 'y0'] = Ys.min()
        d.loc[i, 'y1'] = Ys.min()
        d.loc[i, 'y2'] = Ys.max()
        d.loc[i, 'y3'] = Ys.max()

    rotate_angles[path] = d.avg_ang[withinRange(d.avg_ang)].mean()

    for col in ["x0", "y0", "x1", "y1","x2", "y2", "x3", "y3"]:
        d[col] = d[col].astype(int)

    d[["x0", "y0", "x1", "y1","x2", "y2", "x3", "y3", "text"]].to_csv(r'IndianDataset/box/'+file_name+'.txt', header=None, index=None, sep=',')


#     df = df.append(d, ignore_index=True)

In [ ]:
rotate_angles

In [ ]:
with open('rotate_angles.json', 'w') as fp: 
    json.dump(rotate_angles, fp) # save rotated angle if needed

In [ ]:
def rotate_img_and_save(path, angle):
    image = cv2.imread(path)
    rotatedImage = imutils.rotate(image, angle=angle)
    cv2.imwrite('rotated\\'+path.split('\\')[-1],rotatedImage)

In [ ]:
for path in image_paths:
    rotate_img_and_save(path, rotate_angles[path]) # rotate images according to detected tilted angle if needed. And then rerun OCR cell above with path to rotated images